In [47]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
import pandas as pd
full_data = pd.read_csv("../../data/processed/historical/train/lcl_data.csv")
df_100K = full_data.sample(100000, random_state=0)
df_100K.to_csv("../../data/processed/historical/train/lcl_data_100K.csv", index=False)

# Load Data

In [49]:
import torch
import numpy as np
import random
RANDOM_STATE = 0
torch.manual_seed(RANDOM_STATE)
torch.use_deterministic_algorithms(True)
g = torch.Generator()
g.manual_seed(RANDOM_STATE)

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

In [50]:
from pathlib import Path
from opensynth.data_modules.lcl_data_module import LCLDataModule
import pytorch_lightning as pl

import matplotlib.pyplot as plt

data_path = Path("../../data/processed/historical/train/lcl_data_100K.csv")
stats_path = Path("../../data/processed/historical/train/mean_std.csv")
outlier_path = Path("../../data/processed/historical/train/outliers.csv")

dm = LCLDataModule(data_path=data_path, stats_path=stats_path, batch_size=100000, n_samples=100000)
dm.setup()

In [51]:
import torch
from opensynth.models.faraday import FaradayVAE
vae_model = torch.load("vae_model.pt")
vae_model.eval()

FaradayVAE(
  (encoder): Encoder(
    (encoder_layers): Sequential(
      (0): Linear(in_features=50, out_features=512, bias=True)
      (1): GELU(approximate='none')
      (2): Linear(in_features=512, out_features=256, bias=True)
      (3): GELU(approximate='none')
      (4): Linear(in_features=256, out_features=128, bias=True)
      (5): GELU(approximate='none')
      (6): Linear(in_features=128, out_features=64, bias=True)
      (7): GELU(approximate='none')
      (8): Linear(in_features=64, out_features=32, bias=True)
      (9): GELU(approximate='none')
      (10): Linear(in_features=32, out_features=16, bias=True)
    )
  )
  (decoder): Decoder(
    (latent): Linear(in_features=18, out_features=16, bias=True)
    (latent_activations): GELU(approximate='none')
    (decoder_layers): Sequential(
      (0): Linear(in_features=16, out_features=32, bias=True)
      (1): GELU(approximate='none')
      (2): Linear(in_features=32, out_features=64, bias=True)
      (3): GELU(approximate='no

In [52]:
from opensynth.models.faraday.gaussian_mixture.prepare_gmm_input import encode_data_for_gmm

next_batch = next(iter(dm.train_dataloader()))
input_tensor = encode_data_for_gmm(data=next_batch, vae_module=vae_model)
input_data = input_tensor.detach().numpy()
n_samples = len(input_tensor)

In [53]:
N_COMPONENTS = 200
REG_COVAR = 1e-4
EPOCHS = 25
IDX = 0
CONVERGENCE_TOL = 1e-2


In [54]:
input_tensor.shape, input_tensor[0][0]

(torch.Size([100000, 18]), tensor(0.4973, grad_fn=<SelectBackward0>))

# Init GMM

In [55]:
from opensynth.models.faraday.new_gmm import gmm_utils

labels_, means_, responsibilities_ = gmm_utils.initialise_centroids(
        X=input_data, n_components=N_COMPONENTS
    )
print(labels_.dtype, responsibilities_.dtype, means_.dtype)

torch.float32 torch.float32 torch.float32


In [56]:
from opensynth.models.faraday.new_gmm.train_gmm import initialise_gmm_params

gmm_init_params = initialise_gmm_params(
    X=input_data,
    n_components = N_COMPONENTS,
    reg_covar=REG_COVAR,
)
print(gmm_init_params["precision_cholesky"][IDX][0][0])
print(gmm_init_params["weights"].sum())

tensor(5.5201)
tensor(1.)


# Torch Lightning Batch Learning

In [57]:
from torch.utils.data import Dataset, DataLoader
from pytorch_lightning import LightningDataModule
class CustomDataset(Dataset):
    def __init__(self, data_tensor: torch.Tensor):
        self.data = data_tensor
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        return self.data[idx]
    
class CustomDataModule(LightningDataModule):
    def __init__(self, data_tensor: torch.Tensor, batch_size: int):
        super().__init__()
        self.data_tensor = data_tensor
        self.batch_size = batch_size
    def setup(self, stage=""):
        self.custom_ds = CustomDataset(self.data_tensor)
    def train_dataloader(self):
        return DataLoader(self.custom_ds, batch_size=self.batch_size, shuffle=False, generator=g, worker_init_fn=seed_worker)
    
custom_dm = CustomDataModule(data_tensor=input_tensor, batch_size=25000)
custom_dm.setup(stage="")

In [58]:
from opensynth.models.faraday.new_gmm.new_gmm_model import GaussianMixtureLightningModule, GaussianMixtureModel
gmm_module = GaussianMixtureModel(
    num_components=N_COMPONENTS,
    num_features = input_data.shape[1],
    reg_covar=REG_COVAR,
    print_idx=IDX
)
gmm_module.initialise(gmm_init_params)
print(f"Initial prec chol: {gmm_module.precision_cholesky[IDX][0][0]}. Initial mean: {gmm_module.means[IDX][0]}")

gmm_lightning_module = GaussianMixtureLightningModule(
    gmm_module = gmm_module,
    vae_module = vae_model,
    num_components = gmm_module.num_components,
    num_features = gmm_module.num_features,
    reg_covar = gmm_module.reg_covar,
    convergence_tolerance = CONVERGENCE_TOL,
    sync_on_batch=False
)
trainer = pl.Trainer(max_epochs=EPOCHS, accelerator="cpu", deterministic=True )
trainer.fit(gmm_lightning_module, custom_dm)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/charlotte.avery/.virtualenvs/OpenSynth-BNsxhSIM/lib/python3.11/site-packages/pytorch_lightning/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
/Users/charlotte.avery/.virtualenvs/OpenSynth-BNsxhSIM/lib/python3.11/site-packages/pytorch_lightning/core/optimizer.py:182: `LightningModule.configure_optimizers` returned `None`, this fit will run with no optimizer

  | Name                      | Type                    | Params | Mode 
------------------------------------------------------------------------------
0 | gmm_module                | GaussianMixtureModel    | 0      | train
1 | vae_module                | FaradayVAE              | 402 K  | eval 
2 | weight_metric             | WeightsMetric           | 0      | train
3 | mean_metric               | MeansMetric             | 0      | train
4 | precision_ch

Initial prec chol: 5.520060062408447. Initial mean: -0.3777449131011963
Epoch 0: 100%|██████████| 4/4 [00:02<00:00,  1.71it/s, v_num=97]Local weights at rank: 0 - means: 0.0227, -0.3274
Reduced weights, means, covar: 0.0227,-0.3274, 0.0145
NLL:  tensor(3.2226)
Epoch 1: 100%|██████████| 4/4 [00:02<00:00,  1.94it/s, v_num=97]Local weights at rank: 0 - means: 0.0206, -0.3119
Reduced weights, means, covar: 0.0206,-0.3119, 0.0070
NLL:  tensor(2.7505)
Epoch 2: 100%|██████████| 4/4 [00:01<00:00,  2.16it/s, v_num=97]Local weights at rank: 0 - means: 0.0192, -0.3112
Reduced weights, means, covar: 0.0192,-0.3112, 0.0049
NLL:  tensor(2.5507)
Epoch 3: 100%|██████████| 4/4 [00:01<00:00,  2.05it/s, v_num=97]Local weights at rank: 0 - means: 0.0219, -0.3172
Reduced weights, means, covar: 0.0219,-0.3172, 0.0037
NLL:  tensor(2.4509)
Epoch 4: 100%|██████████| 4/4 [00:02<00:00,  1.88it/s, v_num=97]Local weights at rank: 0 - means: 0.0233, -0.3243
Reduced weights, means, covar: 0.0233,-0.3243, 0.0031
NLL:

In [59]:
gmm_lightning_module.gmm_module.means

tensor([[-0.3372, -1.6866,  0.7808,  ..., -1.8457,  7.6356,  3.6006],
        [ 0.0866, -4.2852, -1.5447,  ..., -3.4242,  8.5332,  2.4288],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.2197, -3.5089,  0.3784,  ..., -1.8151,  1.7423,  2.8159],
        [-1.0260, -1.2639,  0.1915,  ..., -1.4918,  6.2066,  2.4581]])

In [60]:
ligthning_sum_components = gmm_lightning_module.gmm_module.means.sum(axis=1)
len(ligthning_sum_components[ligthning_sum_components==0])

91

In [63]:
from opensynth.models.faraday.new_gmm.new_gmm_model import GaussianMixtureLightningModule, GaussianMixtureModel
gmm_module = GaussianMixtureModel(
    num_components=N_COMPONENTS,
    num_features = input_data.shape[1],
    reg_covar=REG_COVAR,
    print_idx=IDX
)
gmm_module.initialise(gmm_init_params)
print(f"Initial prec chol: {gmm_module.precision_cholesky[IDX][0][0]}. Initial mean: {gmm_module.means[IDX][0]}")

gmm_lightning_module_sync_on_batch = GaussianMixtureLightningModule(
    gmm_module = gmm_module,
    vae_module = vae_model,
    num_components = gmm_module.num_components,
    num_features = gmm_module.num_features,
    reg_covar = gmm_module.reg_covar,
    convergence_tolerance = CONVERGENCE_TOL,
    sync_on_batch=True
)
trainer = pl.Trainer(max_epochs=EPOCHS, accelerator="cpu", deterministic=True )
trainer.fit(gmm_lightning_module_sync_on_batch, custom_dm)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/charlotte.avery/.virtualenvs/OpenSynth-BNsxhSIM/lib/python3.11/site-packages/pytorch_lightning/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
/Users/charlotte.avery/.virtualenvs/OpenSynth-BNsxhSIM/lib/python3.11/site-packages/pytorch_lightning/core/optimizer.py:182: `LightningModule.configure_optimizers` returned `None`, this fit will run with no optimizer

  | Name                      | Type                    | Params | Mode 
------------------------------------------------------------------------------
0 | gmm_module                | GaussianMixtureModel    | 0      | train
1 | vae_module                | FaradayVAE              | 402 K  | eval 
2 | weight_metric             | WeightsMetric           | 0      | train
3 | mean_metric               | MeansMetric             | 0      | train
4 | precision_ch

Initial prec chol: 5.520060062408447. Initial mean: -0.3777449131011963
Epoch 8: 100%|██████████| 4/4 [00:01<00:00,  2.12it/s, v_num=99]


## SK-learn batch learning

In [64]:
from sklearn.mixture import GaussianMixture
init_weights = gmm_init_params["weights"]
init_means = gmm_init_params["means"]

skgmm = GaussianMixture(n_components=N_COMPONENTS, covariance_type='full', tol=CONVERGENCE_TOL, max_iter=EPOCHS, random_state=0, means_init = init_means, weights_init=init_weights, warm_start=True, verbose=1)

dl = custom_dm.train_dataloader()
next_batch = next(iter(dl))
for batch_num, batch_data in enumerate(dl):
    print("Batch number: ", batch_num)
    input_data = batch_data.detach().numpy()
    n_samples = len(input_tensor)
    skgmm.fit(input_data)


Batch number:  0
Initialization 0
  Iteration 10
  Iteration 20
Initialization did not converge.


/Users/charlotte.avery/.virtualenvs/OpenSynth-BNsxhSIM/lib/python3.11/site-packages/sklearn/mixture/_base.py:270: ConvergenceWarning: Best performing initialization did not converge. Try different init parameters, or increase max_iter, tol, or check for degenerate data.
  warnings.warn(


Batch number:  1
Initialization 0
Initialization converged.
Batch number:  2
Initialization 0
Initialization converged.
Batch number:  3
Initialization 0
Initialization converged.


In [65]:
skgmm.means_

array([[ -0.30618054,  -1.47463266,   0.68778584, ...,  -1.60301522,
          7.70324733,   3.42070754],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       ...,
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  0.77502952, -17.0128566 ,  -2.80012655, ...,  -9.32744549,
          6.78769812,   2.81103486],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ]])

In [66]:
sklearn_sum_components = skgmm.means_.sum(axis=1)
len(sklearn_sum_components[sklearn_sum_components==0])

146

# Compare

In [ ]:
IDX = 0

In [67]:
df_compare_means = pd.DataFrame()
df_compare_means["skgmm"] = skgmm.means_[IDX]
df_compare_means["lightning"] = gmm_lightning_module.gmm_module.means[IDX]
df_compare_means["lightning_sync_on_batch"] = gmm_lightning_module_sync_on_batch.gmm_module.means[IDX]
df_compare_means

,skgmm,lightning,lightning_sync_on_batch
0,-0.306181,-0.337200,-0.336467
1,-1.474633,-1.686565,-1.686929
2,0.687786,0.780775,0.780130
3,-0.474157,-0.542568,-0.541846
4,0.231004,0.240737,0.240994
5,-0.001993,-0.010027,-0.009520
6,0.834502,0.906717,0.906535
7,0.653401,0.762841,0.763397
8,0.017217,0.045150,0.045366
9,-1.285122,-1.667377,-1.663710


In [68]:
df_compare_covar = pd.DataFrame()
df_compare_covar["skgmm"] = skgmm.covariances_[IDX][0]
df_compare_covar["lightning"] = gmm_lightning_module.gmm_module.covariances.detach().numpy()[IDX][0]
df_compare_covar

,skgmm,lightning
0,0.003163,0.002201
1,0.004195,0.006886
2,-0.006199,-0.003824
3,0.005487,0.003254
4,-0.000483,0.000313
5,0.000880,0.000568
6,-0.005268,-0.003171
7,-0.003719,-0.005101
8,-0.001326,-0.000934
9,0.014676,0.013735


In [69]:
df_compare_pre_chol = pd.DataFrame()
df_compare_pre_chol["skgmm"] = skgmm.precisions_cholesky_[IDX][0]
df_compare_pre_chol["lightning"] = gmm_lightning_module.gmm_module.precision_cholesky.detach().numpy()[IDX][0]
df_compare_pre_chol

,skgmm,lightning
0,17.781950,21.315332
1,-3.511831,-10.792260
2,21.642546,13.888551
3,-2.777278,-6.007632
4,1.039080,3.488479
5,-28.402608,-15.339210
6,4.845971,2.664494
7,18.016088,7.985599
8,3.086015,-0.438242
9,-40.384889,-21.287474


In [70]:
df_compare_weights = pd.DataFrame()
df_compare_weights["skgmm"] = skgmm.weights_[:10]
df_compare_weights["lightning"] = gmm_lightning_module.gmm_module.weights.detach().numpy()[:10]
df_compare_weights

,skgmm,lightning
0,2.467866e-02,1.684311e-02
1,8.881784e-20,9.620389e-03
2,8.881784e-20,4.768371e-12
3,8.881784e-20,4.768371e-12
4,4.638766e-03,8.733222e-03
5,8.881784e-20,1.103993e-02
6,8.881784e-20,7.561954e-03
7,8.881784e-20,4.768371e-12
8,8.881784e-20,4.768371e-12
9,8.881784e-20,1.225538e-02
